In [1]:
# import dependencies
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

warnings.filterwarnings("ignore")

### Pre-processing

In [2]:
df_108 = pd.read_csv(("../Resources/108justice.csv"))
df_108.tail()

,justice,term,direction,precedentAlteration,issueArea,lawType,caseOriginState
2241,108,2017,1,1,2,3,0
2242,108,2017,2,0,1,2,55
2243,108,2017,1,0,9,9,0
2244,108,2017,2,0,2,6,0
2245,108,2017,1,0,1,6,0


In [ ]:
# df_108["direction"] = np.where(df_108["direction"] == 1, 0, 1)
# df_108.tail(5)

In [3]:
target_108 = df_108["direction"]
target_names = ["conservative", "liberal"]

In [4]:
data_108 = df_108.drop(["direction", "justice"], axis=1)
# feature_names = data_108.columns
data_108.head()

,term,precedentAlteration,issueArea,lawType,caseOriginState
0,1991,0,9,5,12
1,1991,0,8,3,0
2,1991,0,2,3,0
3,1991,0,1,0,0
4,1991,0,9,4,0


In [18]:
# One-hot encoding
term_df = pd.get_dummies(data_108["term"], prefix='term')
precAlt_df = pd.get_dummies(data_108["precedentAlteration"], prefix='precedentAlteration')
issueArea_df = pd.get_dummies(data_108["issueArea"], prefix='issueArea')
lawType_df = pd.get_dummies(data_108["lawType"], prefix='lawType')
caseOriginState_df = pd.get_dummies(data_108["caseOriginState"], prefix='caseOriginState')

precAlt_df.head()

,precedentAlteration_0,precedentAlteration_1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [19]:
oneHot_features = pd.concat([term_df, precAlt_df], axis=1)
oneHot_features = pd.concat([oneHot_features, issueArea_df], axis=1)
oneHot_features = pd.concat([oneHot_features, lawType_df], axis=1)
oneHot_features = pd.concat([oneHot_features, caseOriginState_df], axis=1)
oneHot_features.head()

,term_1991,term_1992,term_1993,term_1994,term_1995,term_1996,term_1997,term_1998,term_1999,term_2000,...,caseOriginState_49,caseOriginState_50,caseOriginState_51,caseOriginState_52,caseOriginState_53,caseOriginState_55,caseOriginState_56,caseOriginState_57,caseOriginState_58,caseOriginState_59
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
X = oneHot_features
feature_names = oneHot_features.columns
y = target_108

### Evaluating different classifiers

In [33]:
clf_lr = LogisticRegression(random_state=1)
clf_rf = RandomForestClassifier(random_state=1, n_estimators=200)
clf_nb = GaussianNB()
clf_svm = SVC(kernel='linear', probability=True)

print('5-fold cross validation:\n')

labels = ["Logistic Regression", "Random Forest", "Naive Bayes", "Linear SVM"]

for clf, label in zip([clf_lr, clf_rf, clf_nb, clf_svm], labels):
    scores = cross_val_score(clf, X, y, cv=5, 
                             scoring='f1', n_jobs=-1)
    print("F1: %0.3f [%s]" % (scores.mean(), label))


5-fold cross validation:

F1: 0.729 [Logistic Regression]
F1: 0.728 [Random Forest]
F1: 0.133 [Naive Bayes]
F1: 0.736 [Linear SVM]


In [34]:
# using all but random forest (since score was low) for voting classifier estimators
v_clf_hard = VotingClassifier(estimators = [(labels[0], clf_lr),
                                            (labels[1], clf_rf),
                                            (labels[3], clf_svm)],
                                            voting = 'hard')

In [37]:
v_clf_soft = VotingClassifier(estimators = [(labels[0], clf_lr),
                                            (labels[1], clf_rf),
                                            (labels[3], clf_svm)],
                                            voting = 'soft')

In [38]:
new_labels = ["Logistic Regression", "Random Forest", "Linear SVM", 
              "Voting_Classifier_Hard", "Voting_Classifier_Soft"]

for (clf, label) in zip([clf_lr, clf_rf, clf_svm, v_clf_hard, v_clf_soft], new_labels):
    scores = cross_val_score(clf, X, y,cv=5,
                             scoring='f1', n_jobs=-1)
    print("F1: %0.3f [%s]" % (scores.mean(), label))

F1: 0.729 [Logistic Regression]
F1: 0.728 [Random Forest]
F1: 0.736 [Linear SVM]
F1: 0.734 [Voting_Classifier_Hard]
F1: 0.739 [Voting_Classifier_Soft]


### Training the SVM model

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [61]:
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [62]:
# Model Accuracy
print('Train Acc: %.3f' % model.score(X_train, y_train))
print('Test Acc: %.3f' % model.score(X_test, y_test))

Train Acc: 0.691
Test Acc: 0.664


In [63]:
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

conservative       0.68      0.94      0.79       378
     liberal       0.44      0.09      0.15       184

   micro avg       0.66      0.66      0.66       562
   macro avg       0.56      0.52      0.47       562
weighted avg       0.60      0.66      0.58       562



In [64]:
confusion_matrix(predictions, y_test)
predictions[:15], y_test[:15].ravel()

(array([2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64),
 array([2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64))